# 解析環境を構築する

実験を行う解析環境を構築します。  <br>
本ノートブックで提供している手順は下記です <br>

 1. NII-RCOSが提供している解析基盤を利用した解析環境の構築手順
 2. OCS（学認クラウドオンデマンド構築サービス） [注1] を利用した解析環境の構築手順
 3. ポータブル版VCP( Virtual Cloud Provider ) [注2] を利用した解析環境の構築手順

このうち、2. 3. の手順では、ユーザ様に用意いただいたクラウド環境へと解析環境を構築する手順となりますので、<br>
解析環境を利用するための準備をお願いいたします。<br>
OCS/VCPを利用するためのクラウド環境の対象は[リンク](https://nii-gakunin-cloud.github.io/ocs-docs/VCPSDK-doc/#provider)の通りです。<br>

注1: https://cloud.gakunin.jp/ocs/<br>
注2: https://github.com/nii-gakunin-cloud/ocs-vcp-portable<br>

## 手順2. と 手順 3.についての補記

手順2. では以下の構成にて、解析環境の構築を行います。<br>

![手順2の構成](./images/RF002003_ocs-figure_01.png)

SaaSとして提供されているOCSを利用することで手順2.を実現いたします。<br>
そのため、OCS利用に伴った利用申請などを行っていただく必要がございます。<br>

<br>
一方、ポータブル版VCPを利用する手順3.では以下の構成にて解析環境の構築を行います<br>

![手順3の構成](./images/RF002003_ocs-figure_04.png)

この手順では、ユーザ様にポータブル版VCPを運用いただくことにより、<br>
OCSのユーザ登録を行っていただくことのみで、OCSのテンプレートを利用した解析環境の構築が可能です。<br>
<br>

## 1. OCSテンプレートの取得をする
OCSのテンプレートのディレクトリとファイルを ResearchFlow内に展開します。

In [ ]:
!git clone https://github.com/NII-DG/ocs-templates-dg.git ocs-templates -b feature/build_env
!rm -rf ocs-templates/.git

## 2. 解析環境を構築する環境を準備する
クラウドサービスによっては事前に解析環境をデプロイする環境を準備する必要があります。<br>
以下のコードセルを実行し、出力されたリンクから準備を行ってください。<br>
<br>
準備の完了後に、本ノートブックへと戻りタスクを継続してください。<br>

In [ ]:
# 環境の準備
import os
import panel as pn
import sys
import traceback
from IPython.display import display

sys.path.append('../../../../..')
from library.task_director import TaskDirector
from library.utils.widgets import MessageBox
from library.utils.config import path_config, message as msg_config
from library.utils.setting import CloudService
from library.utils.html.button import create_button

# 本ファイルのファイル名
script_file_name = os.path.splitext(os.path.basename(__file__))[0]
notebook_name = script_file_name+'.ipynb'

DEFAULT_WIDTH = 600

class ExperimentEnvCreator(TaskDirector):

    def __init__(self, working_path: str) -> None:
        """ExperimentEnvCreator コンストラクタ

        Args:
            working_path (str): [実行Notebookファイルパス]
        """
        super().__init__(working_path, notebook_name)

        # フォームボックス
        self._form_box = pn.WidgetBox()
        self._form_box.width = 900

        # 可変フォームボックス
        self._template_form_box = pn.WidgetBox()
        self._template_form_box.width = 900

        # メッセージ用ボックス
        self._msg_output = MessageBox()
        self._msg_output.width = 900

    @TaskDirector.task_cell('1')
    def generateFormScetion(self):
        # タスク開始によるサブフローステータス管理JSONの更新
        self.doing_task()

        # フォーム定義
        self.set_ocs_template_selector()

        # フォーム表示
        pn.extension()
        form_section = pn.WidgetBox()
        form_section.append(self._form_box)
        form_section.append(self._msg_output)
        display(form_section)

    def set_ocs_template_selector(self):
        self._form_box.clear()
        self._template_form_box.clear()

        self.cloud_service = CloudService()

        self.cloud_service_list = pn.widgets.Select(
            name=msg_config.get('select_cloud_service', 'cloud_service_title'),
            options=self.cloud_service.get_names(),
            disabled_options=self.cloud_service.get_disabled_names(),
            size=4,
            width=600
        )
        self.cloud_service_list.param.watch(self._cloud_service_select_callback, 'value')
        self._form_box.append(self.cloud_service_list)

        # 選択中の行(一番上の行)を選択したときのイベントを発生させる
        self._cloud_service_select_callback(None)

    def _cloud_service_select_callback(self, event):
        try:
            selected = self.cloud_service_list.value
            self.set_link_form(selected)
        except Exception:
            message = f'## [INTERNAL ERROR] : {traceback.format_exc()}'
            self._msg_output.update_error(message)
            return

    def set_link_form(self, service_name):
        try:
            self._msg_output.clear()
            self._template_form_box.clear()

            id = self.cloud_service.get_id(service_name)
            path = self.cloud_service.get_path(service_name)
            description = self.cloud_service.get_description(service_name)

            if id == 'S001':
                # mdx
                md = pn.pane.Markdown(description)
                link = path_config.get_ocs_template_dir() + path
                self._template_form_box.extend(
                    pn.Column(md, self.create_link_button(link))
                )
                self._form_box.append(self._template_form_box)

            else:
                # その他
                md = pn.pane.Markdown(description)
                self._template_form_box.extend(
                    pn.Column(md)
                )
                self._form_box.append(self._template_form_box)

        except Exception:
            message = f'## [INTERNAL ERROR] : {traceback.format_exc()}'
            self._msg_output.update_error(message)
            return

    def create_link_button(self, link):
        """OCS-Templateへのボタンpanel.HTMLオブジェクトの取得
        Returns:
            [panel.pane.HTML]: [HTMLオブジェクト]
        """
        button_width = 500
        link_button = pn.pane.HTML()
        link_button.object = create_button(
            url=link,
            msg=msg_config.get('select_cloud_service', 'go_link'),
            target='_blank',
            button_width=f'{button_width}px'
        )
        link_button.width = button_width
        return link_button


ExperimentEnvCreator(working_path=os.path.abspath('__file__')).generateFormScetion()

## 3. 解析環境の構築手順を選択する
リサーチフローで準備されている構築手順を選択いただきます。<br>
以下のコードセルを実行し、出力されたリンクから構築を行ってください。<br>
<br>
構築の完了後に、本ノートブックへと戻りタスクを継続してください。<br>

In [ ]:
# 構築手順選択
import os
import sys
import traceback

import panel as pn
from IPython.display import display

sys.path.append('../../../../..')
from library.task_director import TaskDirector
from library.utils.widgets import MessageBox
from library.utils.config import path_config, message as msg_config
from library.utils.setting import OCSTemplate
from library.utils.html.button import create_button

script_file_name = "build_experiment_environment"
notebook_name = script_file_name+'.ipynb'

class ExperimentEnvBuilder(TaskDirector):

    def __init__(self, working_path:str) -> None:
        """ExperimentEnvBuilder コンストラクタ

        Args:
            working_path (str): [実行Notebookファイルパス]
        """
        super().__init__(working_path, notebook_name)

        # フォームボックス
        self._form_box = pn.WidgetBox()
        self._form_box.width = 900

        # 可変フォームボックス
        self._template_form_box = pn.WidgetBox()
        self._template_form_box.width = 900

        # メッセージ用ボックス
        self._msg_output = MessageBox()
        self._msg_output.width = 900

    def set_ocs_template_selector(self):
        self._form_box.clear()
        self._template_form_box.clear()

        self.ocs_template = OCSTemplate()
        options = []
        options.extend(self.ocs_template.get_name())

        self.ocs_template_list = pn.widgets.Select(
                name=msg_config.get('select_ocs_template', 'ocs_template_title'),
                options=options,
                disabled_options=self.ocs_template.get_disabled_ids(),
                size=4,
                width=600
            )

        self.ocs_template_list.param.watch(self._ocs_template_select_callback, 'value')
        self._form_box.append(self.ocs_template_list)

    def _ocs_template_select_callback(self, event):
        try:
            self.selected = self.ocs_template_list.value
            self.set_templatelink_form()
        except Exception:
            message = f'## [INTERNAL ERROR] : {traceback.format_exc()}'
            self._msg_output.update_error( message )
            return

    def set_templatelink_form(self):
        try:
            self._msg_output.clear()
            self._template_form_box.clear()

            self.construction_procedure_id = self.ocs_template.get_id(self.selected)
            self.template_path = self.ocs_template.get_template_path(self.selected)
            if self.template_path is None:
                raise Exception('Don\'t Get Path of OCS Template')

            if self.construction_procedure_id == "T001":
                # 解析基盤をそのまま利用
                md = pn.pane.Markdown( msg_config.get('select_ocs_template', 'use_computing_service') )
                self._template_form_box.extend(
                    pn.Column( md,self.get_ocs_template_button_object() )
                )
                self._form_box.append(self._template_form_box)

            elif self.construction_procedure_id == "T999":
                # ポータブル版VCCを利用して構築する。
                message = msg_config.get('select_ocs_template', 'use_portable_vcc') \
                            + '<br>' \
                            + msg_config.get('select_ocs_template', 'success')

                self.template_link = self.template_path

                md = pn.pane.Markdown( message )
                self._template_form_box.extend(
                    pn.Column( md,self.get_ocs_template_button_object() )
                )
                self._form_box.append(self._template_form_box)

            else:
                # OCSテンプレートを利用して構築する。
                message = msg_config.get('select_ocs_template', 'use_ocs_template') \
                            + '<br>' \
                            + msg_config.get('select_ocs_template', 'success')

                self.template_link =  path_config.get_ocs_template_dir() + self.template_path

                md = pn.pane.Markdown( message )
                self._template_form_box.extend(
                    pn.Column( md,self.get_ocs_template_button_object() )
                )
                self._form_box.append(self._template_form_box)

        except Exception:
            message = f'## [INTERNAL ERROR] : {traceback.format_exc()}'
            self._msg_output.update_error( message )
            return

    #########################
    #  ocs-template link    #
    #########################
    def get_ocs_template_button_object(self)-> pn.pane.HTML:
        """OCS-Templateへのボタンpanel.HTMLオブジェクトの取得
        Returns:
            [panel.pane.HTML]: [HTMLオブジェクト]
        """
        button_width = 500
        ocs_template_link_button = pn.pane.HTML()
        ocs_template_link_button.object = create_button(
            url=self.template_link,
            msg=msg_config.get('select_ocs_template', 'go_template_link'),
            target='_blank',
            button_width=f'{button_width}px'
        )
        ocs_template_link_button.width = button_width
        return ocs_template_link_button

    @TaskDirector.task_cell("1")
    def generateFormScetion(self):
        # タスク開始によるサブフローステータス管理JSONの更新
        self.doing_task()

        # フォーム定義
        self.set_ocs_template_selector()

        # フォーム表示
        pn.extension()
        form_section = pn.WidgetBox()
        form_section.append(self._form_box)
        form_section.append(self._msg_output)
        display(form_section)


ExperimentEnvBuilder(working_path=os.path.abspath('__file__')).generateFormScetion()

## 4. 構築手順を保存する
構築を行った手順をGakunin RDMへと同期可能なフォルダへと配置します。<br>
構築を行った手順を保存する場合は、以下のコードセルを実行してください。<br>

In [ ]:
!SAVE_DIR_NAME=`date "+%Y%m%d%H%M%S"`
!mkdir -p ~/data/plan/build_experiment_environment/${SAVE_DIR_NAME}
!cp -pr ocs-templates/TheLittlestJupyterHub/ ~/data/plan/build_experiment_environment/${SAVE_DIR_NAME}/

## 5.Gakunin RDMに実行結果を同期
構築手順をGakunin RDMに同期します。<br>
構築手順を保存する場合は、以下のコードセルを実行してください。<br>

In [ ]:
# 保存する
import os
import sys

import panel as pn

sys.path.append('../../../../..')
from library.task_director import TaskDirector
from library.utils.config import path_config

script_file_name = "build_experiment_environment"
notebook_name = script_file_name+'.ipynb'

class ExperimentEnvBuilder(TaskDirector):

    def __init__(self, working_path:str) -> None:
        """ExperimentEnvBuilder コンストラクタ

        Args:
            working_path (str): [実行Notebookファイルパス]
        """
        super().__init__(working_path, notebook_name)

    TaskDirector.task_cell("2")
    def completed_task(self):
        # タスク実行の完了情報を該当サブフローステータス管理JSONに書き込む
        self.done_task()

        # フォーム定義
        save_target_path = os.path.join( self._abs_root_path, path_config.DATA, path_config.PLAN, "build_experiment_environment")
        source = self.get_sync_source(save_target_path)
        self.define_save_form(source)

        # フォーム表示
        pn.extension()
        form_section = pn.WidgetBox()
        form_section.append(self.save_form_box)
        form_section.append(self.save_msg_output)
        display(form_section)

    def get_sync_source(self, search_directory :str):
        source = []
        for root, dirs, files in os.walk(search_directory):
            for filename in files:
                if filename.endswith(".ipynb"):
                    path = os.path.join(root, filename)
                    source.append(path)
        return source

ExperimentEnvBuilder(working_path=os.path.abspath('__file__')).completed_task()

## サブフローメニューへ

サブフローメニューへアクセスするボタンを表示します

In [ ]:
# サブフローメニューへアクセスするボタンを表示
import os
import sys
sys.path.append('../../../../..')
from library.task_director import TaskDirector

script_file_name = "build_experiment_environment"
notebook_name = script_file_name+'.ipynb'
TaskDirector(os.path.abspath('__file__'), notebook_name).return_subflow_menu()